In [1]:
import pandas as pd

In [2]:
# 1. Importar datos desde un archivo Excel
# Cambia 'ruta_al_archivo.xlsx' por la ruta real de tu archivo
ruta_archivo = "ATENEO BASE PLANA (3).xlsx"
df = pd.read_excel(ruta_archivo, header=3)

In [3]:
for col in df.select_dtypes(include=["object", "string"]).columns:
    df[col] = df[col].map(lambda x: x.strip() if isinstance(x, str) else x)

In [4]:
df.loc[:, "dosis_aux"] = df["Dosis3"].map(
    lambda x: 30 if x == 28 else x
)


In [5]:
df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)


In [6]:
df_filter_tipo = df[df["TIPO"].isin(["ME", "MINV"])]


In [ ]:
# df_filter_tipo.loc[:, "dosis_aux"] = df_filter_tipo["Dosis3"].map(
#     lambda x: 30 if x == 28 else x
# )


In [7]:
# 3. Filtrar los datos del laboratorio "Siegfried"
df_siegfried = df_filter_tipo[df_filter_tipo["Laboratorio"].str.strip() == "Siegfried"]
df_siegfried = df_siegfried[df_siegfried["TIPO"].isin(["ME", "MINV"])]
df_siegfried = df_siegfried.drop_duplicates()
df_siegfried.reset_index(drop=True, inplace=True)
df_siegfried.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 98 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   TIPO                         833 non-null    object 
 1   TIPO-MERCADO                 833 non-null    object 
 2   CODIGO                       833 non-null    object 
 3   CT                           833 non-null    object 
 4   CT4                          833 non-null    object 
 5   Molécula(s)                  833 non-null    object 
 6   Corp                         833 non-null    object 
 7   Xplora                       833 non-null    object 
 8   Laboratorio                  833 non-null    object 
 9   Marca                        833 non-null    object 
 10  Presentación                 833 non-null    object 
 11  FF                           833 non-null    object 
 12  Lab - Marca                  833 non-null    object 
 13  Lab - Presentacion  

In [8]:
# Elimina columnas de la 22 a la 88 (índices basados en 0)
df_filter_tipo = df_filter_tipo.drop(df_filter_tipo.iloc[:, 22:89], axis=1)


In [ ]:
# df_filter_tipo[[
#     "TIPO", "FF", "dosis_aux", "Concentration Type & Factor", "Molécula(s)"
# ]].dtypes

In [ ]:
# df_siegfried[
#     ["TIPO", "FF", "Dosis3", "Concentration Type & Factor", "Molécula(s)"]
# ].dtypes


In [9]:

# Ahora hacemos merge sobre el df completo
df_completo_type_dosis_mol = df_filter_tipo.merge(
    df_siegfried[
        [
            "TIPO",
            "FF",
            "Concentration Type & Factor",
            "dosis_aux",
            "Molécula(s)",
            "Precio PVP",
        ]
    ],
    left_on=["TIPO", "FF", "dosis_aux", "Concentration Type & Factor", "Molécula(s)"],
    right_on=["TIPO", "FF", "dosis_aux", "Concentration Type & Factor", "Molécula(s)"],
    how="left",
    suffixes=("", "_Siegfried"),
)


In [ ]:
df_completo_type_dosis_mol.info()

In [10]:
# Renombrar la columna
df_completo_type_dosis_mol.rename(columns={"Precio PVP_Siegfried": "Precio_Siegfried"}, inplace=True)

# Corregir la condición de la asignación
df_completo_type_dosis_mol.loc[
    df_completo_type_dosis_mol["Laboratorio"] == "Siegfried", "Precio_Siegfried"
] = None

# Guardar el DataFrame como archivo CSV
# df_completo_type_dosis_mol.to_csv("df_completo_type_dosis_mol.csv", index=False)

In [11]:
df_completo_type_dosis_mol['dif_vs_sieg'] = (
    df_completo_type_dosis_mol['Precio_Siegfried'] / df_completo_type_dosis_mol['Precio PVP'] - 1
)

In [12]:
print(df_completo_type_dosis_mol[df_completo_type_dosis_mol['dif_vs_sieg'].notna()][['Precio_Siegfried', 'Precio PVP', 'dif_vs_sieg']].head())


    Precio_Siegfried  Precio PVP  dif_vs_sieg
26          19093.08    20555.00    -0.071122
27          13512.67    20555.00    -0.342609
28          19093.08    20555.00    -0.071122
29          13512.67    20555.00    -0.342609
36           8704.98     9974.25    -0.127255


In [ ]:
df_completo_type_dosis_mol.info()

In [13]:
df_completo_type_dosis_mol = df_completo_type_dosis_mol.drop_duplicates()
df_completo_type_dosis_mol.reset_index(drop=True, inplace=True)


In [16]:
df_completo_type_dosis_mol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20370 entries, 0 to 20369
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   TIPO                         20370 non-null  object 
 1   TIPO-MERCADO                 20370 non-null  object 
 2   CODIGO                       20370 non-null  object 
 3   CT                           20368 non-null  object 
 4   CT4                          20370 non-null  object 
 5   Molécula(s)                  20370 non-null  object 
 6   Corp                         20370 non-null  object 
 7   Xplora                       20370 non-null  object 
 8   Laboratorio                  20370 non-null  object 
 9   Marca                        20370 non-null  object 
 10  Presentación                 20370 non-null  object 
 11  FF                           20364 non-null  object 
 12  Lab - Marca                  20370 non-null  object 
 13  Lab - Presentaci

In [17]:
df_completo_type_dosis_mol.to_excel("df_completo_type_dosis_mol_ff.xlsx", index=False)

In [ ]:

# # Ahora hacemos merge sobre el df completo
# df_completo_type_dosis_mol = df_filter_tipo.merge(
#     df_siegfried[["TIPO", "FF", "Concentration Type & Factor", "Dosis3", "Molécula(s)", "Precio PVP"]],
#     on=["TIPO", "FF", "Concentration Type & Factor", "Dosis3", "Molécula(s)"],
#     how="left",
#     suffixes=("", "_Siegfried")
# )


In [14]:
df_particular = df_completo_type_dosis_mol[
    (
        df_completo_type_dosis_mol["CODIGO"].isin(["3126302", "3407801"])
        & (df_completo_type_dosis_mol["TIPO"] == 'ME')
    )
]


In [ ]:
df_particular


In [15]:
fila_1 = df_particular.iloc[0]
fila_2 = df_particular.iloc[1]

    # Comparar los valores de cada columna
comparacion = fila_1 == fila_2

In [ ]:
comparacion

In [ ]:
df_completo_type_dosis_mol.duplicados.head()

In [ ]:
# 1. Establecer la primera fila como encabezado
#df.columns = df.iloc[0]  # Establecer la primera fila como encabezado (índice 0)

# 2. Eliminar la fila que se usó como encabezado
#df = df.drop(0).reset_index(drop=True)
df.head()

In [ ]:
df.columns

In [ ]:
# 2. Seleccionar solo las columnas necesarias
# Cambia los nombres de las columnas por los que están en tu archivo Excel
columnas_necesarias = ["Laboratorio", "Concentration Type & Factor", "Dosis3", "Molécula(s)", "Precio PVP"]
df_short = df[columnas_necesarias]

In [ ]:
df_short = df_short.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
for col in df_short.select_dtypes(include=["object", "string"]).columns:
    df_short[col] = df_short[col].map(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
df_type_dosis_mol = df_short.merge(
    df_siegfried[["Concentration Type & Factor", "Dosis3", "Molécula(s)", "Precio PVP"]],
    on=["Concentration Type & Factor", "Dosis3", "Molécula(s)"],
    how="left",
    suffixes=("", "_Siegfried")
)

In [ ]:
df_type_dosis_mol.describe()

In [ ]:
df_type_dosis_mol.info()

In [ ]:
# 4. Realizar un merge para agregar los precios de Siegfried al DataFrame original
df_type_mol = df_short.merge(
    df_siegfried[["Concentration Type & Factor", "Dosis3", "Molécula(s)", "Precio PVP"]],
    on=["Concentration Type & Factor", "Molécula(s)"],
    how="left",
    suffixes=("", "_Siegfried")
)

In [ ]:
df_type_mol.describe()

In [ ]:
df_type_mol.info()

In [ ]:
df_type_mol.to_csv("tipo_mol.csv", index=False)
df_type_dosis_mol.to_csv("tipo_molecula_dosis.csv", index=False)


In [ ]:
df_type_dosis_mol[df_type_dosis_mol["Molécula(s)"] == "Aciclovir"].head()

In [ ]:
df_siegfried.describe()

In [ ]:
df_short[df_short["Laboratorio"].str.strip() == "Siegfried"].count()

In [ ]:
# 5. Renombrar la columna para claridad
df_type_mol.rename(columns={"Precio PVP_Siegfried": "Precio_Siegfried"}, inplace=True)
df_type_dosis_mol.rename(columns={"Precio PVP_Siegfried": "Precio_Siegfried"}, inplace=True)

# 6. Eliminar los precios de Siegfried para sus propias filas
df_type_mol.loc[df["Laboratorio"] == "Siegfried", "Precio_Siegfried"] = None
df_type_dosis_mol.loc[df["Laboratorio"] == "Siegfried", "Precio_Siegfried"] = None


In [ ]:
# Mostrar los primeros registros para verificar
print(df_type_mol.head())

In [ ]:
df_type_mol_conprecio = df_type_mol[df_type_mol["Precio_Siegfried"].notna()]

In [ ]:
# 7. Exportar los datos a un nuevo archivo Excel (opcional)
ruta_salida = "resultado.xlsx"
df.to_excel(ruta_salida, index=False)
